In [ ]:
import batoid
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv

In [ ]:
telescope = batoid.Optic.fromYaml("HSC.yaml")

In [ ]:
fig1 = plt.figure(figsize=(12, 11))
ax1 = fig1.add_subplot(111)
telescope.draw2d(ax1, c='k')

fig2 = plt.figure(figsize=(12, 11))
ax2 = fig2.add_subplot(111, projection='3d')
telescope.draw3d(ax2, c='k')

ipv.figure(width=700, height=500)
telescope.draw3d(ipv, color='black')

x = np.array([-2.5, -1.5, 1.5, 2.5])

for vx, c in zip(np.deg2rad([-0.7, 0, 0.7]), ['#377eb8', '#ff7f00', '#4daf4a']):
    rays = batoid.RayVector(x, 0, 0, vx, 0, -np.sqrt(1-vx**2), wavelength=500e-9)
    rays.propagate(-20)
    tF = telescope.traceFull(rays)
    batoid.drawTrace2d(ax1, tF, c=c)
    batoid.drawTrace3d(ax2, tF, c=c)
    batoid.drawTrace3d(ipv, tF, color=c)
ax1.set_aspect('equal')
ipv.show()

In [ ]:
fig1 = plt.figure(figsize=(12, 11))
ax1 = fig1.add_subplot(111)
telescope['HSC'].draw2d(ax1, c='k')
ax1.set_xlim(ax1.get_xlim())
ax1.set_ylim(ax1.get_ylim())

fig2 = plt.figure(figsize=(12, 11))
ax2 = fig2.add_subplot(111, projection='3d')
telescope['HSC'].draw3d(ax2, c='k')
ax2.set_xlim(ax2.get_xlim())
ax2.set_ylim(ax2.get_ylim())
ax2.set_zlim(ax2.get_zlim())

ipv.figure(width=700, height=500)
telescope.draw3d(ipv, color='black')

x = np.array([-2.5, -1.5, 1.5, 2.5])

for vx, c in zip(np.deg2rad([-0.7, 0, 0.7]), ['#377eb8', '#ff7f00', '#4daf4a']):
    rays = batoid.RayVector(x, 0, 0, vx, 0, -np.sqrt(1-vx**2), wavelength=500e-9)
    rays.propagate(-20)
    tF = telescope.traceFull(rays)
    batoid.drawTrace2d(ax1, tF, c=c)
    batoid.drawTrace3d(ax2, tF, c=c)
    batoid.drawTrace3d(ipv, tF, color=c)
ipv.xlim(-0.5, 0.5)
ipv.ylim(-0.5, 0.5)
ipv.zlim(13.4, 15.4)
ipv.show()